In [ ]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
arg_date = '2021-11-15'

In [3]:
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

In [4]:
arg_date_dt

datetime.date(2021, 11, 14)

In [5]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('deutsche-boerse-xetra-pds')

In [6]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], '%Y-%m-%d').date() >= arg_date_dt]

In [15]:
objects = objects[0:10]

In [16]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [17]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [18]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = df_all.append(df, ignore_index=True)

In [19]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [20]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2021-11-15,08:00,19.910,19.910,19.910,19.910,3628
1,DE000A0DJ6J9,2021-11-15,08:00,49.920,49.920,49.560,49.680,6358
2,DE000A0D6554,2021-11-15,08:00,14.710,14.710,14.620,14.700,14162
3,DE000A0D9PT0,2021-11-15,08:00,197.400,197.450,196.850,197.200,4316
4,DE000A0JL9W6,2021-11-15,08:00,65.000,65.000,65.000,65.000,1853
...,...,...,...,...,...,...,...,...
27780,DE000A3CQ7F4,2021-11-15,09:59,17.520,17.520,17.520,17.520,135
27781,LU2356314745,2021-11-15,09:59,15.040,15.040,15.040,15.040,54
27782,US3696043013,2021-11-15,09:59,93.920,93.920,93.600,93.600,913
27783,DE000VTSC017,2021-11-15,09:59,51.300,51.300,51.300,51.300,112


In [21]:
df_all.dropna(inplace=True)

In [22]:
df_all.shape

(27785, 8)

## Get opening price per ISIN and day

In [23]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [24]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2021-11-15,08:00,19.91,19.91,19.91,19.91,3628,19.91
5302,AT0000A0E9W5,2021-11-15,08:13,19.90,19.93,19.90,19.93,2750,19.91
5606,AT0000A0E9W5,2021-11-15,08:14,19.93,19.98,19.93,19.98,469,19.91
6322,AT0000A0E9W5,2021-11-15,08:17,19.98,20.00,19.98,20.00,740,19.91
6906,AT0000A0E9W5,2021-11-15,08:19,19.95,19.95,19.93,19.93,585,19.91
7655,AT0000A0E9W5,2021-11-15,08:22,19.94,19.94,19.94,19.94,250,19.91
7863,AT0000A0E9W5,2021-11-15,08:23,19.92,19.92,19.92,19.92,1,19.91
8102,AT0000A0E9W5,2021-11-15,08:24,19.92,19.93,19.90,19.93,2385,19.91
8316,AT0000A0E9W5,2021-11-15,08:25,19.90,19.90,19.90,19.90,818,19.91
8531,AT0000A0E9W5,2021-11-15,08:26,19.90,19.94,19.90,19.94,2248,19.91


## Get closing price per ISIN and day

In [25]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [26]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,AT0000A0E9W5,2021-11-15,08:00,19.91,19.91,19.91,19.91,3628,19.91,20.4
5302,AT0000A0E9W5,2021-11-15,08:13,19.90,19.93,19.90,19.93,2750,19.91,20.4
5606,AT0000A0E9W5,2021-11-15,08:14,19.93,19.98,19.93,19.98,469,19.91,20.4
6322,AT0000A0E9W5,2021-11-15,08:17,19.98,20.00,19.98,20.00,740,19.91,20.4
6906,AT0000A0E9W5,2021-11-15,08:19,19.95,19.95,19.93,19.93,585,19.91,20.4
7655,AT0000A0E9W5,2021-11-15,08:22,19.94,19.94,19.94,19.94,250,19.91,20.4
7863,AT0000A0E9W5,2021-11-15,08:23,19.92,19.92,19.92,19.92,1,19.91,20.4
8102,AT0000A0E9W5,2021-11-15,08:24,19.92,19.93,19.90,19.93,2385,19.91,20.4
8316,AT0000A0E9W5,2021-11-15,08:25,19.90,19.90,19.90,19.90,818,19.91,20.4
8531,AT0000A0E9W5,2021-11-15,08:26,19.90,19.94,19.90,19.94,2248,19.91,20.4


## Aggregations

In [27]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [28]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT00000FACC2,2021-11-15,8.3800,8.3300,8.2000,8.3800,1501
1,AT0000606306,2021-11-15,27.6600,27.8400,27.6000,28.0000,1648
2,AT0000609607,2021-11-15,12.8000,12.9600,12.6600,12.9600,2820
3,AT0000644505,2021-11-15,112.0000,112.4000,110.4000,112.4000,359
4,AT0000652011,2021-11-15,40.2500,41.0100,40.2500,41.0100,108
...,...,...,...,...,...,...,...
2765,XS2265369731,2021-11-15,9.4455,9.4455,9.4455,9.4455,0
2766,XS2265370234,2021-11-15,18.2450,18.2450,18.2450,18.2450,0
2767,XS2284324667,2021-11-15,34.0020,34.0020,34.0020,34.0020,0
2768,XS2314659447,2021-11-15,8.5440,8.5990,8.5440,8.5990,951


## Percent Change Prev Closing

In [29]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [30]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price
0,AT00000FACC2,2021-11-15,8.3800,8.3300,8.2000,8.3800,1501,NaN
1,AT0000606306,2021-11-15,27.6600,27.8400,27.6000,28.0000,1648,NaN
2,AT0000609607,2021-11-15,12.8000,12.9600,12.6600,12.9600,2820,NaN
3,AT0000644505,2021-11-15,112.0000,112.4000,110.4000,112.4000,359,NaN
4,AT0000652011,2021-11-15,40.2500,41.0100,40.2500,41.0100,108,NaN
...,...,...,...,...,...,...,...,...
2765,XS2265369731,2021-11-15,9.4455,9.4455,9.4455,9.4455,0,NaN
2766,XS2265370234,2021-11-15,18.2450,18.2450,18.2450,18.2450,0,NaN
2767,XS2284324667,2021-11-15,34.0020,34.0020,34.0020,34.0020,0,NaN
2768,XS2314659447,2021-11-15,8.5440,8.5990,8.5440,8.5990,951,NaN


In [31]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [32]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [33]:
df_all = df_all.round(decimals=2)

In [34]:
df_all = df_all[df_all.Date >= arg_date]

In [35]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-11-15,8.38,8.33,8.20,8.38,1501,NaN
1,AT0000606306,2021-11-15,27.66,27.84,27.60,28.00,1648,NaN
2,AT0000609607,2021-11-15,12.80,12.96,12.66,12.96,2820,NaN
3,AT0000644505,2021-11-15,112.00,112.40,110.40,112.40,359,NaN
4,AT0000652011,2021-11-15,40.25,41.01,40.25,41.01,108,NaN
...,...,...,...,...,...,...,...,...
2765,XS2265369731,2021-11-15,9.45,9.45,9.45,9.45,0,NaN
2766,XS2265370234,2021-11-15,18.24,18.24,18.24,18.24,0,NaN
2767,XS2284324667,2021-11-15,34.00,34.00,34.00,34.00,0,NaN
2768,XS2314659447,2021-11-15,8.54,8.60,8.54,8.60,951,NaN


## Write to S3

In [36]:
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [40]:
!pip install pyarrow

     |████████████████████████████████| 19.1 MB 2.7 MB/s            


In [41]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('xetra-target')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='xetra-target', key='xetra_daily_report_20211118_232908.parquet')

## Reading the uploaded file

In [42]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20211118_232908.parquet
xetra_daily_report_20211118_233011.parquet


In [47]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20211118_233011.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [48]:
df_report.shape

(419611, 8)